<a href="https://colab.research.google.com/github/Elisshaze/into2ML/blob/main/NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install split-folders

In [2]:
import torch
import torchvision
import splitfolders
import torch.utils.data as data
import torch.utils.tensorboard as tb
from torchvision import datasets, transforms as T
import pandas as pd
import torch.utils.tensorboard as tb
import torch.optim as optim
from torch.optim import lr_scheduler 
# This enables running tensorboard in the notebook
%load_ext tensorboard
import torch.nn as nn
BATCH_SIZE = 128

In [28]:
cuda = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! tar -zxvf /content/drive/MyDrive/into2ML/dataset.tar.gz

In [6]:
!mkdir -p dataset/test/unknown
!mv dataset/test/*.png dataset/test/unknown/

In [7]:
num_classes = 10

In [8]:
transform_train = T.Compose([T.Resize(256), 
                       T.CenterCrop(224), 
                       T.ToTensor(),
])
transform_test = T.Compose([
                            T.Resize(224),
                            T.ToTensor(),
])

In [9]:
splitfolders.ratio("./dataset/train", output="./dataset/train_val", seed=1337, ratio=(.8, .2), group_prefix=None) # default values

Copying files: 12415 files [00:48, 257.65 files/s]


In [10]:
trDataSet = torchvision.datasets.ImageFolder('./dataset/train_val/train/', transform = transform_train)
vlDataSet = torchvision.datasets.ImageFolder('./dataset/train_val/val/', transform = transform_test)
tsDataSet = torchvision.datasets.ImageFolder('./dataset/test/', transform = transform_test )

Normalization code taken from https://pytorch.org/vision/stable/models.html, as I am not using the pretrained model in this case.

In [11]:
means = []
stds = []
for img, _ in (trDataSet):
    means.append(torch.mean(img))
    stds.append(torch.std(img))

mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))
print (mean, std)

normalize_tr = T.Normalize(mean=mean, std=std)

tensor(0.1713) tensor(0.1216)


In [12]:
for img, _ in (tsDataSet):
    means.append(torch.mean(img))
    stds.append(torch.std(img))

mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))
print (mean, std)

normalize_ts = T.Normalize(mean=mean, std=std)

tensor(0.1685) tensor(0.1169)


In [13]:
transform_train = T.Compose([
                       T.ToTensor(),
                       normalize_tr,
])
transform_test = T.Compose([
                            T.ToTensor(),
                            normalize_ts,
])

In [14]:
trDataSet = torchvision.datasets.ImageFolder('./dataset/train_val/train/', transform = transform_train)
vlDataSet = torchvision.datasets.ImageFolder('./dataset/train_val/val/', transform = transform_test)
tsDataSet = torchvision.datasets.ImageFolder('./dataset/test/', transform = transform_test )

In [15]:
trDataLoader = torch.utils.data.DataLoader(trDataSet, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
vlDataLoader = torch.utils.data.DataLoader(vlDataSet, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
tsDataLoader = torch.utils.data.DataLoader(tsDataSet, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

In [ ]:
model = torchvision.models.resnet50(pretrained = False, progress = True)
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
model = model.to(cuda)
print (model)

In [17]:
#Performance evaluation
class ClassificationMetrics:

  # Constructor takes the number of classes
  def __init__(self, num_classes=10):
    self.num_classes = num_classes
    # Initialize a confusion matrix
    self.C = torch.zeros(num_classes, num_classes) 
    self.C = self.C.to(cuda)

  # Update the confusion matrix with the new scores
  def add(self, yp, yt):
    # yp: 1D tensor with predictions
    # yt: 1D tensor with ground-truth targets

    with torch.no_grad(): # We require no computation graph
      yt = yt.to(cuda)
      yp = yp.to(cuda)
      self.C+=(yt*self.C.shape[1]+yp).bincount(minlength=self.C.numel()).view(self.C.shape).float()

  def clear(self):
    # We set the confusion matrix to zero
    self.C.zero_()

  # Computes the global accuracy
  def acc(self):
    return self.C.diag().sum().item()/self.C.sum()

  # Computes the class-averaged accuracy
  def mAcc(self):
    return (self.C.diag()/self.C.sum(-1)).mean().item()

  # Computers the class-averaged Intersection over Union
  def mIoU(self):
    return (self.C.diag()/(self.C.sum(0)+self.C.sum(1)-self.C.diag())).mean().item()

  # Returns the confusion matrix
  def confusion_matrix(self):
    return self.C

In [18]:
def train_one_epoch(model, loss_func, metric_tracker, dataloader, optimizer, epoch, tblog=None):
  # Function to perform one training epoch
  #
  # model: PyTorch model to train
  # loss_func: training loss to be used
  # metric_tracker: classification metric tracker
  # dataloader: dataloader with the training data
  # optimizer:PyTorch optimizer to use
  # epoch: current epoch number
  # tblog: tensorboard summary writer

  
  # We set the model in training mode. Some layers, typically involving a 
  # stochastic component, behave differently if run in evaluation vs training 
  # mode (e.g., dropout, batch normalization, etc.). For this reason we need
  # to specify the modality.
  model.train()

  # Clear the metric tracker
  metric_tracker.clear()

  # We iterate over each mini-batch by tracking the iteration number
  for i,(X,yt) in enumerate(dataloader):
    X = X.to(cuda)
    yt = yt.to(cuda)
    # Set gradients to zero
    optimizer.zero_grad()

    # Forward pass
    # We assume that the model outputs a 2D tensor, where the second dimension
    # indexes class labels
    Y = model(X)

    # Compute the loss 
    loss = loss_func(Y, yt)

    # The index of the largest output along the second dimension gives the predicted
    # class label
    y = Y.argmax(-1)

    # Track evaluation metrics
    metric_tracker.add(y, yt)

    # If a tensorboard summary writer is available we track the loss value
    # and the metrics of each iteration
    if tblog:
      tblog.add_scalar('train/loss', loss.item(), epoch*len(dataloader)+i)

    # Perform the backward pass to compute gradients
    loss.backward()
    val_loss = loss.item()

    # Update the parameters
    optimizer.step()
    return val_loss

In [19]:
def validate(model, metric_tracker, dataloader):
  # Function to perform a validation of the current model
  #
  # model: PyTorch model to train
  # metric_tracker: classification metric tracker
  # dataloader: dataloader with the validatoin data

  # We set the model in evaluation mode. Some layers, typically involving a 
  # stochastic compponent, behave differently if run in evaluation vs training 
  # mode (e.g., dropout, batch normalization, etc.). For this reason we need
  # to specify the modality.
  model.eval()

  # Clear the metric tracker
  metric_tracker.clear()

  # This informs PyTorch that no gradient computation is needed and, therefore,
  # it should not track the computation graph
  with torch.no_grad(): 
    # We iterate over each mini-batch by tracking the iteration number
    for i,(X,yt) in enumerate(dataloader):
        X = X.to(cuda)
        yt = yt.to(cuda)
        model.to(cuda)
      # Forward pass
      # We assume that the model outputs a 2D tensor, where the second dimension
      # indexes class labels
        Y = model(X)

      # The index of the largest output along the second dimension gives the predicted
      # class label
        y = Y.argmax(-1)

      # Track evaluation metrics
        metric_tracker.add(y,yt)

In [20]:
def train(model, trDataLoader, vlDataLoader, optimizer, lr_scheduler, num_epochs, tblog=None):
  # Training function
  #
  # model: PyTorch model to train (we assume it exposes a num_classes function)
  # trDataLoader: dataloader for training data
  # vlDataLoader: dataloader for validation data
  # optimizer: the PyTorch optimizer to use
  # lr_scheduler: the PyTorch scheduler for the learning rate
  # num_epochs: number of epochs to train the model
  # tblog: tensorboard logger

  # We create the loss function to be used for training
  loss_func = nn.CrossEntropyLoss()

  # We create the performance metric tracker
  metric_tracker = ClassificationMetrics(num_classes)

  for epoch in range(1,num_epochs+1):

    print("-- EPOCH {}/{} -------------------------\n".format(epoch, num_epochs))

    # Train for one epoch
    val_loss= train_one_epoch(model, loss_func, metric_tracker, trDataLoader, optimizer, epoch, tblog)

    # Print metrics accumulated over the epoch
    print("\tTRAIN | acc: {:.4f} | mAcc: {:.4f} | mIoU: {:.4f}".format(
        metric_tracker.acc(), metric_tracker.mAcc(), metric_tracker.mIoU()
    ))
    
    # Track the metrics in the tensorboard log
    if tblog:
      tblog.add_scalar('train/acc', metric_tracker.acc(), epoch)
      tblog.add_scalar('train/mAcc', metric_tracker.mAcc(), epoch)
      tblog.add_scalar('train/mIoU', metric_tracker.mIoU(), epoch)
      print("VAL_LOSS: ", val_loss)
      #tblog.add_scalar('train/lr', scheduler._last_lr, epoch)

    # Evaluate the current model
    validate(model, metric_tracker, vlDataLoader)

    # Print metrics over the validation set
    print("\tEVAL  | acc: {:.4f} | mAcc: {:.4f} | mIoU: {:.4f}\n".format(
        metric_tracker.acc(), 
        metric_tracker.mAcc(), metric_tracker.mIoU()
    ))

    # Track the metrics in the tensorboard log
    if tblog:
      tblog.add_scalar('val/acc', metric_tracker.acc(), epoch)
      tblog.add_scalar('val/mAcc', metric_tracker.mAcc(), epoch)
      tblog.add_scalar('val/mIoU', metric_tracker.mIoU(), epoch)

    # Update the learning rate according to the scheduler

    lr_scheduler.step(val_loss)

In [21]:
def test(model, dataloader):
  # Function to validate the model and print the final score
  #
  # model: PyTorch model to test (we assume it exposes a num_classes function)
  # dataloader: dataloader for test data

  # We create the performance metric tracker
  metric_tracker = ClassificationMetrics(num_classes)

  # We run the validation code con the test data and track the performance
  validate(model,metric_tracker, dataloader)

  # Print metrics over the test set
  print("TEST  | acc: {:.4f} | mAcc: {:.4f} | mIoU: {:.4f}\n".format(
      metric_tracker.acc(), 
      metric_tracker.mAcc(), metric_tracker.mIoU()
  ))

  return metric_tracker

# Here we apply the test function to the model we trained before
#test(model, tsDataLoader);

In [22]:
def run_experiment(name, model):

  # We create a simple Stochastic Gradient Descent (SGD) optimizer, with momentum 
  # and weight-decay (i.e. l2 regularization)
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.00001)

  # We add a learning rate scheduler that reduces the learning rate by a factor
  # 0.1 after 10 and 15 epochs, respectively

  #optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum = 0.9)
  #lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [10,15], gamma=0.5)

  
  lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=1, verbose=True)
  #for epoch in range(10):
  # We train our model
  tblog=tb.SummaryWriter("exps/{}".format(name))
  train(model, trDataLoader, vlDataLoader, optimizer, lr_scheduler, num_epochs=40, tblog=tblog)
  #val_loss = validate(...)
  # Note that step should be called after validate()

  # We create the tensorboard logger
 

  

# Run an experiment with an instance of our SimpleMLP with 256 and 128 units in the first and 
# second hidden layers, respectively.
#model = SimpleMLP(784, 256, 128, 10)
#run_experiment('MLP-256-128-Sigmoid', model)

In [ ]:
%tensorboard --logdir exps/resnNet50_1

In [29]:
results={}
name='resnNet50_1'
run_experiment(name, model)
!mkdir -p 'dataset/save/resnNet50_1'
torch.save(model, 'dataset/save/resnet50_1')
results[name]= test(model, tsDataLoader)

-- EPOCH 1/40 -------------------------



RuntimeError: ignored